
# Assignment 3: Text Classification Using the Stanford SST Sentiment Dataset 
## Xi Lin

### Q1: Discuss the dataset in general terms and describe why building a predictive model using this data might be practically useful.  Who could benefit from a model like this? Explain.

In [2]:
#install aimodelshare library
! pip install aimodelshare==0.0.189

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 967.8/967.8 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.0/294.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.8/511.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2

In [1]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/sst2_competition_data-repository:latest') 


Data downloaded successfully.


In [2]:
# Set up X_train, X_test, and y_train_labels objects
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=Warning)

X_train=pd.read_csv("sst2_competition_data/X_train.csv", squeeze=True)
X_test=pd.read_csv("sst2_competition_data/X_test.csv", squeeze=True)

y_train_labels=pd.read_csv("sst2_competition_data/y_train_labels.csv", squeeze=True)

# ohe encode Y data
y_train = pd.get_dummies(y_train_labels)

X_train.head()

0    The Rock is destined to be the 21st Century 's...
1    The gorgeously elaborate continuation of `` Th...
2    Singer/composer Bryan Adams contributes a slew...
3                 Yet the act is still charming here .
4    Whether or not you 're enlightened by any of D...
Name: text, dtype: object

Answer: 
The SST2 dataset is a collection of movie reviews and their corresponding sentiment labels. Building a predictive model using this data can be practically useful for several reasons. First, by training a model to accurately predict sentiment in movie reviews, we can improve our understanding of natural language and how to analyze it. Second, a well-performing model can be adapted to other domains and industries that require sentiment analysis, such as social media, customer reviews, or market research. 

Various stakeholders can benefit from a model like this, including:

1. Movie studios and distributors: A model that can accurately predict movie review sentiment can be useful for gauging audience reactions, understanding public opinion, and making data-driven decisions about marketing and distribution.
2. Social media platforms: These platforms can use sentiment analysis to understand user opinions on various topics, helping to inform content recommendations, advertising, and moderation efforts. 
3. Businesses: Companies can use the model to analyze customer feedback on their products or services, helping them to identify areas of improvement and capitalize on positive sentiment.

### Run at least three prediction models to try to predict the SST sentiment dataset well.
#### 1. Use an Embedding layer and LSTM layers in at least one model

In [3]:
# This preprocessor function makes use of the tf.keras tokenizer

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
import numpy as np

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# preprocessor tokenizes words and makes sure all documents have the same length
def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

print(preprocessor(X_train).shape)
print(preprocessor(X_test).shape)

(6920, 40)
(1821, 40)


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Flatten

# Preprocess the data
X_train_preprocessed = preprocessor(X_train)
X_test_preprocessed = preprocessor(X_test)

# Define the model 
model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(LSTM(32, return_sequences=True, dropout=0.2))
model.add(LSTM(32, dropout=0.2))
model.add(Flatten())
model.add(Dense(2, activation='softmax')) 

# Compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
173/173 [==============================] - 14s 48ms/step - loss: 0.6494 - accuracy: 0.6230 - val_loss: 0.7185 - val_accuracy: 0.5759
Epoch 2/10
173/173 [==============================] - 6s 35ms/step - loss: 0.4971 - accuracy: 0.7648 - val_loss: 0.6322 - val_accuracy: 0.6871
Epoch 3/10
173/173 [==============================] - 9s 54ms/step - loss: 0.3724 - accuracy: 0.8358 - val_loss: 0.5993 - val_accuracy: 0.7341
Epoch 4/10
173/173 [==============================] - 6s 34ms/step - loss: 0.3134 - accuracy: 0.8696 - val_loss: 0.7459 - val_accuracy: 0.6366
Epoch 5/10
173/173 [==============================] - 7s 38ms/step - loss: 0.2603 - accuracy: 0.8873 - val_loss: 0.5803 - val_accuracy: 0.7536
Epoch 6/10
173/173 [==============================] - 9s 50ms/step - loss: 0.2262 - accuracy: 0.9088 - val_loss: 0.6008 - val_accuracy: 0.7355
Epoch 7/10
173/173 [==============================] - 6s 34ms/step - loss: 0.2022 - accuracy: 0.9178 - val_loss: 0.7374 - val_accuracy: 0.716

In [6]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials
    
apiurl="https://rlxjxnoql9.execute-api.us-east-1.amazonaws.com/prod/m" #This is the unique rest api that powers this specific Playground

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [7]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 
#Instantiate Competition

mycompetition= ai.Competition(apiurl)

Your preprocessor is now saved to 'preprocessor.zip'


In [5]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [9]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 5s 28ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 261

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


#### 2. Use an Embedding layer and Conv1d layers in at least one model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Define the model2
model2 = Sequential()
model2.add(Embedding(10000, 16, input_length=40))
model2.add(Conv1D(128, kernel_size=3, activation='relu'))
model2.add(Conv1D(128, kernel_size=3, activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(2, activation='softmax')) 

# Compile the model
model2.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
173/173 [==============================] - 6s 31ms/step - loss: 0.6518 - accuracy: 0.6140 - val_loss: 0.7955 - val_accuracy: 0.3685
Epoch 2/10
173/173 [==============================] - 4s 24ms/step - loss: 0.4932 - accuracy: 0.7619 - val_loss: 0.7465 - val_accuracy: 0.6048
Epoch 3/10
173/173 [==============================] - 3s 17ms/step - loss: 0.3654 - accuracy: 0.8427 - val_loss: 0.6512 - val_accuracy: 0.6936
Epoch 4/10
173/173 [==============================] - 3s 17ms/step - loss: 0.2838 - accuracy: 0.8831 - val_loss: 0.6292 - val_accuracy: 0.7001
Epoch 5/10
173/173 [==============================] - 3s 17ms/step - loss: 0.2261 - accuracy: 0.9088 - val_loss: 0.5841 - val_accuracy: 0.7565
Epoch 6/10
173/173 [==============================] - 5s 30ms/step - loss: 0.1815 - accuracy: 0.9279 - val_loss: 0.6800 - val_accuracy: 0.7276
Epoch 7/10
173/173 [==============================] - 5s 27ms/step - loss: 0.1472 - accuracy: 0.9444 - val_loss: 0.6585 - val_accuracy: 0.7565

In [12]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [13]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
prediction_column_index=model2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 8ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 262

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


#### 3. Use transfer learning with glove embeddings for at least one of these models

In [14]:
# Download Glove embedding matrix weights (Might take 10 mins or so!)
! wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2023-04-17 02:40:52--  http://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/wordvecs/glove.6B.zip [following]
--2023-04-17 02:40:52--  https://nlp.stanford.edu/data/wordvecs/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip [following]
--2023-04-17 02:40:53--  https://downloads.cs.stanford.edu/nlp/data/wordvecs/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182753 (822M) [app

In [47]:
! unzip glove.6B.zip 

Archive:  glove.6B.zip
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [48]:
import numpy as np

def load_glove_embeddings(file_path, word_index, embedding_dim):
    embeddings_index = {}
    with open('glove.6B.100d.txt', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            coeffs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coeffs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

# Load pre-trained GloVe embeddings
glove_file = 'glove.6B.100d.txt'  # Replace with the path to the extracted GloVe file
embedding_dim = 100
embedding_matrix = load_glove_embeddings(glove_file, tokenizer.word_index, embedding_dim)


In [52]:
# Define the model3 
model3 = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=40, weights=[embedding_matrix], trainable=False),
    LSTM(32),
    Flatten(),
    Dense(2, activation='softmax')
])


# Compile the model
model3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model3.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 9s 34ms/step - loss: 0.6185 - accuracy: 0.6510 - val_loss: 0.6915 - val_accuracy: 0.6279
Epoch 2/10
173/173 [==============================] - 3s 18ms/step - loss: 0.5219 - accuracy: 0.7430 - val_loss: 0.6664 - val_accuracy: 0.6763
Epoch 3/10
173/173 [==============================] - 3s 18ms/step - loss: 0.4771 - accuracy: 0.7686 - val_loss: 0.6406 - val_accuracy: 0.6908
Epoch 4/10
173/173 [==============================] - 3s 18ms/step - loss: 0.4563 - accuracy: 0.7859 - val_loss: 0.4380 - val_accuracy: 0.8331
Epoch 5/10
173/173 [==============================] - 5s 27ms/step - loss: 0.4290 - accuracy: 0.8004 - val_loss: 0.5482 - val_accuracy: 0.7551
Epoch 6/10
173/173 [==============================] - 5s 30ms/step - loss: 0.4142 - accuracy: 0.8076 - val_loss: 0.5728 - val_accuracy: 0.7334
Epoch 7/10
173/173 [==============================] - 3s 19ms/step - loss: 0.3954 - accuracy: 0.8185 - val_loss: 0.3910 - val_accuracy: 0.8548

In [54]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model3, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model3.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [55]:
#Submit Model 3: 


#-- Generate predicted y values (Model 3)
prediction_column_index=model3.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model3.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 3s 27ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 277

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


#### 4. Discuss which models performed better and point out relevant hyper-parameter values for successful models.

In [44]:
# Compare two or more models
data=mycompetition.compare_models([1, 2, 3], verbose=1)
mycompetition.stylize_compare(data)

,Model_1_Layer,Model_1_Shape,Model_1_Params,Model_2_Layer,Model_2_Shape,Model_2_Params,Model_3_Layer,Model_3_Shape,Model_3_Params
0,Embedding,"[None, 40, 16]",160000.000000,Embedding,"[None, 40, 16]",160000,Embedding,"[None, 40, 16]",160000.000000
1,Flatten,"[None, 640]",0.000000,LSTM,"[None, 40, 32]",6272,LSTM,"[None, 40, 256]",279552.000000
2,Dense,"[None, 2]",1282.000000,LSTM,"[None, 32]",8320,Flatten,"[None, 10240]",0.000000
3,None,None,nan,Flatten,"[None, 32]",0,Dense,"[None, 2]",20482.000000
4,None,None,nan,Dense,"[None, 2]",66,None,None,nan


In [45]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

The basic LSTM and embedding layer model performed the best with an accuracy of 0.81. All models had similar hyperparameters in terms of epochs and batch_size. The differences in their architecture led to the observed differences in their performance. The LSTM model can capture longer-range dependencies in the input sequences, making it more suitable for sentiment analysis tasks where the context of the whole sentence matters.

The 1D convnets model, on the other hand, focuses more on detecting local patterns, which might not be as effective in capturing the overall sentiment. However, it still performed relatively well with an accuracy of 0.79.

The transfer learning model with GloVe embeddings had an accuracy of 0.80, which is also a good result. Pre-trained word embeddings, such as GloVe, can often improve the performance of models as they provide a more robust representation of words by leveraging the knowledge learned from large-scale text data. In this case, the GloVe-based model's performance was slightly lower than the LSTM model but higher than the 1D convnets model.

In conclusion, the basic LSTM and embedding layer model performed the best among the three models with the given hyperparameters. However, it is important to note that the differences in accuracy are relatively small, and further hyperparameter tuning or architecture adjustments might lead to better results for any of the models.

### After you submit your first three models, describe your best model with your team via your team slack channel
#### 1. Fit and submit up to three more models after learning from your team.

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense,Flatten,Bidirectional

# Preprocess the data
X_train_preprocessed = preprocessor(X_train)
X_test_preprocessed = preprocessor(X_test)
model4 = Sequential()
model4.add(Embedding(10000, 32, input_length=40))
model4.add(Bidirectional(LSTM(32)))  
model4.add(Flatten())
model4.add(Dense(2, activation='softmax'))

model4.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model4.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 16s 50ms/step - loss: 0.6441 - acc: 0.6254 - val_loss: 1.1695 - val_acc: 0.1777
Epoch 2/10
173/173 [==============================] - 8s 48ms/step - loss: 0.4474 - acc: 0.7943 - val_loss: 0.5824 - val_acc: 0.7305
Epoch 3/10
173/173 [==============================] - 5s 31ms/step - loss: 0.3206 - acc: 0.8633 - val_loss: 0.4872 - val_acc: 0.7970
Epoch 4/10
173/173 [==============================] - 9s 50ms/step - loss: 0.2466 - acc: 0.8961 - val_loss: 0.5890 - val_acc: 0.7348
Epoch 5/10
173/173 [==============================] - 6s 37ms/step - loss: 0.2010 - acc: 0.9214 - val_loss: 0.7471 - val_acc: 0.6705
Epoch 6/10
173/173 [==============================] - 5s 31ms/step - loss: 0.1616 - acc: 0.9384 - val_loss: 0.5143 - val_acc: 0.8027
Epoch 7/10
173/173 [==============================] - 10s 58ms/step - loss: 0.1372 - acc: 0.9500 - val_loss: 0.8903 - val_acc: 0.6705
Epoch 8/10
173/173 [==============================] - 6s 34ms/step 

In [8]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model4, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model4.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [9]:
#Submit Model 4: 

#-- Generate predicted y values (Model 4)
prediction_column_index=model4.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 4 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model4.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 2s 9ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 343

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import SimpleRNN, LSTM,Embedding

model5 = Sequential()
model5.add(layers.Embedding(10000, 16, input_length=40))
model5.add(layers.Conv1D(32, 7, activation='relu')) 
model5.add(layers.MaxPooling1D(5)) 
model5.add(layers.Conv1D(128, 3, activation='relu'))
model5.add(layers.GlobalMaxPooling1D())
model5.add(Dense(2, activation='softmax')) 


# Compile the model
model5.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model5.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 4s 14ms/step - loss: 0.6629 - accuracy: 0.6124 - val_loss: 0.9194 - val_accuracy: 0.1488
Epoch 2/10
173/173 [==============================] - 2s 11ms/step - loss: 0.5946 - accuracy: 0.6501 - val_loss: 0.8350 - val_accuracy: 0.4935
Epoch 3/10
173/173 [==============================] - 1s 8ms/step - loss: 0.4622 - accuracy: 0.7780 - val_loss: 0.8156 - val_accuracy: 0.5679
Epoch 4/10
173/173 [==============================] - 1s 8ms/step - loss: 0.3569 - accuracy: 0.8354 - val_loss: 0.6371 - val_accuracy: 0.7081
Epoch 5/10
173/173 [==============================] - 1s 8ms/step - loss: 0.2901 - accuracy: 0.8712 - val_loss: 0.7624 - val_accuracy: 0.6561
Epoch 6/10
173/173 [==============================] - 1s 8ms/step - loss: 0.2380 - accuracy: 0.8945 - val_loss: 0.7568 - val_accuracy: 0.6358
Epoch 7/10
173/173 [==============================] - 1s 7ms/step - loss: 0.1931 - accuracy: 0.9174 - val_loss: 0.9543 - val_accuracy: 0.6012
Epoc

In [12]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model5, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model5.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [13]:
#Submit Model 5: 

#-- Generate predicted y values (Model 5)
prediction_column_index=model5.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 5 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model5.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 0s 2ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 344

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


In [23]:
model6 = Sequential()
model6.add(Embedding(10000, 300, input_length=40))
model6.add(Bidirectional(LSTM(128,return_sequences=True, dropout=0.2)))    
model6.add(Bidirectional(LSTM(64,dropout=0.2))) 
model6.add(Dense(2, activation='softmax'))

model6.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model6.fit(X_train_preprocessed, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
173/173 [==============================] - 100s 510ms/step - loss: 0.5992 - acc: 0.6767 - val_loss: 0.3987 - val_acc: 0.8396
Epoch 2/10
173/173 [==============================] - 72s 414ms/step - loss: 0.3550 - acc: 0.8522 - val_loss: 0.6748 - val_acc: 0.6763
Epoch 3/10
173/173 [==============================] - 70s 405ms/step - loss: 0.2308 - acc: 0.9068 - val_loss: 0.4005 - val_acc: 0.8569
Epoch 4/10
173/173 [==============================] - 70s 403ms/step - loss: 0.1583 - acc: 0.9391 - val_loss: 0.8257 - val_acc: 0.7103
Epoch 5/10
173/173 [==============================] - 72s 417ms/step - loss: 0.1061 - acc: 0.9613 - val_loss: 0.6531 - val_acc: 0.7695
Epoch 6/10
173/173 [==============================] - 67s 390ms/step - loss: 0.0727 - acc: 0.9745 - val_loss: 0.9206 - val_acc: 0.7117
Epoch 7/10
173/173 [==============================] - 72s 417ms/step - loss: 0.0489 - acc: 0.9827 - val_loss: 1.1983 - val_acc: 0.6777
Epoch 8/10
173/173 [==============================] - 

In [24]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model6, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model6.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [25]:
#Submit Model 6: 

#-- Generate predicted y values (Model 6)
prediction_column_index=model6.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 6 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model6.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

57/57 [==============================] - 9s 99ms/step
Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 349

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:2763


#### 2.Discuss results

In [26]:
# Compare two or more models
data=mycompetition.compare_models([4, 5, 6], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_4,model_version_5,model_version_6
0,bootstrap,True,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000,0.000000
2,class_weight,None,None,None,None
3,criterion,gini,gini,gini,gini
4,max_depth,None,3,5,5
5,max_features,auto,auto,auto,auto
6,max_leaf_nodes,None,None,None,None
7,max_samples,None,None,None,None
8,min_impurity_decrease,0.000000,0.000000,0.000000,0.000000
9,min_impurity_split,None,None,None,None


After discuss with my team, the accuracies I obtained for these new models are:

Model 1 (Bidirectional LSTM): accuracy = 0.77
Model 2 (MaxPooling1D for 1D ConvNets): accuracy = 0.71
Model 3 (Bidirectional LSTM and dropout techique): accuracy = 0.78
While these new models achieved good accuracies, they still underperformed compared to my first three models. This could be due to several reasons, such as differences in the model architectures, hyperparameters, or the nature of the dataset.


### Discuss which models you tried and which models performed better and point out relevant hyper-parameter values for successful models.

First set of models:

1. Basic LSTM and embedding layer: accuracy = 0.81
2. Basic embedding layer with 1D convnets: accuracy = 0.79
3. Transfer learning with GloVe embeddings: accuracy = 0.80

Second set of models:

4. Bidirectional LSTM: accuracy = 0.77
5. MaxPooling1D for 1D ConvNets: accuracy = 0.71
6. Bidirectional LSTM and dropout: accuracy = 0.78  

From the results, it is clear that the first set of models performed better than the second set. Among all six models, the best performing one is the basic LSTM and embedding layer model with an accuracy of 0.81. This model used 10 epochs and a batch size of 32.

The LSTM model's performance can be attributed to its ability to capture longer-range dependencies in the input sequences, which is highly relevant for sentiment analysis tasks. The architecture allowed the model to learn complex patterns and maintain context throughout the input sequence.

The other models also performed well but couldn't surpass the LSTM model. The GloVe-based transfer learning model achieved an accuracy of 0.80, while the basic 1D ConvNets model reached 0.79. The second set of models, which includes Bidirectional LSTM and MaxPooling1D for 1D ConvNets, had lower accuracies ranging from 0.71 to 0.78.

To summarize, the best performing model is the basic LSTM and embedding layer model with an accuracy of 0.81. The relevant hyperparameters for this successful model are 10 epochs and a batch size of 32. If want to further improve the performance of your models, I can try hyperparameter tuning, architecture adjustments, cross-validation, and regularization techniques as suggested in the previous response.